In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2
# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp

In [ ]:
import simulated_annealing as sa

In [ ]:
def Weierstrass(x, a=0.5, b=5, N=20):
    # true Weierstrass function is an infinite sum so N = infinity
    a = a or (1+3/2*np.pi)/b
    return sum(a**n * np.cos(b**n * np.pi * x) for n in range(N))
bounds = (-1, 1)

In [ ]:
def _():
    fig, ax = plt.subplots(figsize=(25, 8))
    xs = np.linspace(*bounds, num=2000)
    ax.plot(xs, [Weierstrass(x) for x in xs])
    
    print('global maximum =', Weierstrass(0))
_()

In [ ]:
class hyper_params:
    max_its=100
    
    initial_step=0.3
    step_alter_chunk=10
    
    M=5 # lower seems to be more reliably good
    factor=0.9**5
    T_0=3
    T_min=0.2
    
def optimise(h, seed, quiet=False):
    np.random.seed(seed)
    
    class Optimiser:
        def __init__(self):
            self.step_size = h.initial_step
            self.change_counter = 1
            self.step_size_record = [self.step_size]
            
        def choose_neighbour(self, state, state_record, acceptance_record, temperature):
            # adjust the step size to try to maintain roughly 60% acceptance rate
            chunk = h.step_alter_chunk
            if self.change_counter % chunk == 0:
                acceptance_record = acceptance_record[-chunk:] # only interested in the last few
                average_acceptance = acceptance_record.count(True) / len(acceptance_record)
                x = np.random.uniform(0, 0.1)
                if average_acceptance > 0.7:
                    self.step_size *= 1 + x
                elif average_acceptance < 0.5:
                    self.step_size *= 1 - x
            self.change_counter += 1
            self.step_size_record.append(self.step_size)
            #c = state + np.random.choice([-1, 1]) * self.step_size
            #c = state + np.random.normal(loc=0, scale=self.step_size)
            c = state + np.random.uniform(-1, 1)*self.step_size
            return np.clip(c, *bounds) # ensure not to go out of bounds
        
        def cooling_schedule(self, i):
            return sa.temperature_exponential_decay(i, M=h.M, factor=h.factor, T_0=h.T_0, T_min=h.T_min)

    o = Optimiser()
    best, best_E, rec = sa.simulated_annealing(
        energy_func=lambda x: -Weierstrass(x),
        initial_state=np.random.uniform(*bounds),
        max_its=h.max_its,
        candidate_dist=o.choose_neighbour,
        cooling_schedule=o.cooling_schedule
    )
    if not quiet:
        print('{}/{} accepted'.format(len(rec), h.max_its))
        print('best =', best)
        print('best_E =', best_E)
    return best, best_E, rec, o


In [ ]:
def _():
    best_Es = []
    for i in range(50):
        best, best_E, rec, o = optimise(hyper_params, seed=i, quiet=True)
        best_Es.append(best_E)
    print('average best = {}, var = {}'.format(np.mean(best_Es), np.var(best_Es)))
_()

In [ ]:
_, _, rec, o = optimise(hyper_params, seed=0)

In [ ]:
def _():
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(25, 12))
    xs = np.arange(len(o.step_size_record))
    ax1.plot(xs, o.step_size_record, 'o-')
    ax1.set_xlabel('iteration')
    ax1.set_ylabel('step size')
    ax1.set_yscale('log')
    
    xs = np.arange(hyper_params.max_its)
    ax2.plot(xs, [o.cooling_schedule(i) for i in xs])
    ax2.set_xlabel('iteration')
    ax2.set_ylabel('temperature')
    
    fig.tight_layout()
_()

In [ ]:
def _():
    fig, ax = plt.subplots(figsize=(25, 8))
    xs = np.linspace(*bounds, num=2000)
    ax.plot(xs, [Weierstrass(x) for x in xs])
    best = ax.axvline(x=0, color='k', linestyle='--', animated=True, alpha=0.5)
    current = ax.axvline(x=0, color='r', animated=True)
    fig.tight_layout()
    
    def update(frame):
        v = rec[-1][0] if frame >= len(rec) else rec[frame][0]
        current.set_xdata(v)
        v = min(rec[:frame+1], key=lambda x: x[1])[0]
        best.set_xdata(v)
    import matplotlib as mpl
    import matplotlib.animation
    from IPython.display import display, HTML
    ani = mpl.animation.FuncAnimation(fig, update, frames=np.arange(len(rec) + 40), interval=50)
    display(HTML(ani.to_html5_video()))
    plt.close(fig)
_()